In [7]:
from rhs_tlm import TLM
from plot import plot_distributions_tlm
from total_current import total_current_tlm
from scipy.integrate import solve_ivp
from radauDAE import RadauDAE
from matplotlib import pyplot as plt

import numpy as np
import parameters, grid, initial_conditions, mass_tlm, jacobian_tlm
import time

params = parameters.Params()
grid = grid.Grid(params)
ic = initial_conditions.Initial_Conditions(params, grid)
mass = mass_tlm.Mass(params, grid)
jac = jacobian_tlm.Jac(params)

sol_init = ic.sol_init_tlm
mass = mass.M
jac = jac.jac

tf = 1
method = RadauDAE
rtol = params.rtol
atol = params.atol

dae_fun = lambda t, u: TLM(t, u, pbi)
start_time = time.time()

sol = solve_ivp(fun=dae_fun, t_span=(0.0, tf), y0=sol_init,rtol=rtol, atol=atol, jac_sparsity=jac, method=method, vectorized=False, dense_output=False, mass=mass)

t = sol.t
dae_fun = lambda t, u: TLM(t, u, test1)
sol_init = sol.y[:,-1]

sol = solve_ivp(fun=dae_fun, t_span=(0.0, 10*tf), y0=sol_init, rtol=rtol, atol=atol, jac_sparsity=jac, method=method, vectorized=False, dense_output=True, mass=mass)

sol_init = sol.y[:,-1]
tf_reverse_scan = params.tf_scan
dae_fun = lambda t, u: TLM(t, u, reverse_scan)

sol = solve_ivp(fun=dae_fun, t_span=(0.0, tf_reverse_scan), y0=sol_init, rtol=rtol, atol=atol, jac_sparsity=jac, method=method, vectorized=False, dense_output=True, mass=mass)
t_reverse = sol.t
u_reverse = sol.y

psi_applied_r = params.phi_precondition*np.ones(t_reverse.size) - params
psi_scan_rate*t_reverse
J_reverse = total_current_tlm(t_reverse, u_reverse)[0]
sol_init = sol.y[:,-1]
tf_forward_scan = params.tf_scan
dae_fun = lambda t, u: TLM(t, u, forward_scan)

sol = solve_ivp(fun=dae_fun, t_span=(0.0, tf_forward_scan), y0=sol_init, rtol=rtol, atol=atol, jac_sparsity=jac, method=method, vectorized=False, dense_output=True, mass=mass)
t_forward = sol.t
u_forward = sol.y
psi_applied_f = params.phi_f*np.ones(t_forward.size) + params.psi_scan_rate*t_forward
J_forward = total_current_tlm(t_forward, u_forward)[0]
J_nondim = np.concatenate((J_reverse, J_forward), 0)
psi_scan = np.concatenate((psi_applied_r, psi_applied_f), 0)
u_matrix = np.concatenate((u_reverse, u_forward), 1)
t_vector = np.concatenate((t_reverse, t_forward), 0)
u = sol.y
t = sol.t
x = grid.x
xE = grid.xE
xH = grid.xH

plot_distributions_tlm(x, xE, xH, u[:,-1])
N = params.N
x = grid.x

P = u[0:N+1,-1]
phi = u[N+1:2*N+2,-1]
n = u[2*N+2:3*N+3,-1]
p = u[3*N+3:4*N+4,-1]
plt.figure(0)
plt.plot(x,P)
plt.figure(1)
plt.plot(x,phi)
plt.figure(2)
plt.plot(x,n)
plt.figure(3)
plt.plot(x,p)
plt.figure(4)
plt.plot(psi_applied_r, J_reverse)
#plt.plot(psi_applied_f, J_forward)
plt.xlim([0,40])
plt.ylim([-0.2,0.8])

#print(---␣%s␣seconds␣--- % (time.time() - start_time))

ModuleNotFoundError: No module named 'rhs_tlm'